# **Predikcia rakoviny prsníka s Logistickou Regresiou**

# 1. Úvod

Tento projekt sa zameriava na implementáciu a vyhodnotenie klasifikačného modelu pre diagnostiku rakoviny prsníka. Využíva dataset Breast Cancer Wisconsin Diagnostic, ktorý obsahuje merania bunkových jadier zo vzoriek pacientov. Hlavným cieľom je vytvoriť model, ktorý dokáže spoľahlivo klasifikovať vzorky ako benígne (nezhubné) alebo malígne (zhubné).

V rámci analýzy sa zameriame na Logistickú Regresiu. Tento model bol vybraný pre jeho vysokú presnosť, ktorú potvrdila aj krížová validácia, a predovšetkým pre jeho interpretovateľnosť. To nám umožňuje nielen predpovedať diagnózu, ale aj pochopiť, ktoré medicínske príznaky majú najväčší vplyv na výsledok.

**Dôraz bude kladený na:**

Vysokú presnosť a robustnosť modelu potvrdenú krížovou validáciou.

Minimalizáciu kritických chýb (False Negatives), čo je v medicíne kľúčové.

Jasnú interpretáciu výsledkov z medicínskeho hľadiska, aby mohol slúžiť ako praktický nástroj na podporu rozhodovania.

Konečným výstupom je aj interaktívna Gradio aplikácia, ktorá demonštruje, ako môže byť tento model prakticky využitý lekármi v klinickej praxi.



# 2. Načítanie a prvá analýza dát

Prvým a kľúčovým krokom každého projektu je načítanie dát a ich základná analýza. Cieľom je získať prehľad o štruktúre datasetu, zistiť, či neobsahuje chýbajúce hodnoty, a preskúmať distribúciu našej cieľovej premennej.

**V tejto časti kódu vykonáme nasledovné:**

Načítanie datasetu load_breast_cancer z knižnice scikit-learn do DataFrame.

Kontrola štruktúry dát pomocou metód df.head() a df.info(), aby sme sa uistili, že dáta sú v správnom formáte.

Analýza distribúcie cieľovej premennej (target), ktorá nás informuje o pomere zhubných (0) a nezhubných (1) nádorov. Táto informácia je dôležitá pre výber vhodnej stratégie modelovania.

Vizualizácia distribúcie pomocou stĺpcového grafu, ktorý nám dáva rýchly a jasný prehľad o vyváženosti datasetu.

In [ ]:
# Import potrebných knižníc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer

# Načítanie datasetu Breast Cancer
cancer = load_breast_cancer()
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

# Zobrazíme prvých 5 riadkov a názvy tried
print("Prvých 5 riadkov datasetu:")
print(df.head())
print("\nNázvy tried:", cancer.target_names) # 0 = maligný (zhubný), 1 = benígny (nezhubný)

# Informácie o stĺpcoch a chýbajúcich hodnotách
print("\nInformácie o dátach:")
df.info()

# Kontrola vyváženosti cieľovej premennej
print("\nDistribúcia cieľovej premennej:")
print(df['target'].value_counts())
print("\nPercentuálne rozdelenie:")
print(df['target'].value_counts(normalize=True) * 100)

# Vizualizácia distribúcie
plt.figure(figsize=(6, 4))
sns.countplot(x='target', data=df)
plt.title('Distribúcia cieľovej premennej (0=Zhubný, 1=Nezhubný)')
plt.show()

# 3. Príprava na modelovanie

Po dôkladnej prieskumnej analýze dát prechádzame ku kľúčovému kroku: modelovaniu. Aby sme mohli objektívne zhodnotiť výkon nášho modelu, je nevyhnutné rozdeliť dáta na trénovaciu a testovaciu množinu. Model sa natrénuje len na trénovacích dátach, a jeho výkon sa následne vyhodnotí na testovacej množine, ktorú nikdy predtým nevidel.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 1. Definovanie X a y
X = df.drop('target', axis=1)
y = df['target']

# 2. Rozdelenie dát na trénovaciu a testovaciu množinu (70% tréning, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Trénovacia množina: {X_train.shape[0]} záznamov")
print(f"Testovacia množina: {X_test.shape[0]} záznamov")

# 3. Škálovanie dát (veľmi dôležitý krok pre Logistickú regresiu!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Inštanciovanie a trénovanie modelu
model_lr = LogisticRegression()
model_lr.fit(X_train_scaled, y_train)

print("\nLogistická regresia bola úspešne natrénovaná.")

# 4. Logistická regresia - vyhodnotenie modelu

Po natrénovaní modelu je nevyhnutné objektívne posúdiť, ako dobre funguje model na neznámych dátach. Vyhodnotenie nám poskytne nielen informáciu o celkovej presnosti, ale aj o tom, aké typy chýb model robí, čo je v medicínskom kontexte mimoriadne dôležité.

**V tejto sekcii vypočítame a vizualizujeme kľúčové metriky:**

*Presnosť (Accuracy):* Celkové percento správne klasifikovaných vzoriek.

*Matica zámeny (Confusion Matrix):* Podrobný prehľad, ktorý nám ukáže, koľko vzoriek bolo správne (True) a nesprávne (False) klasifikovaných pre každú triedu.

*False Negative (FN):* Vzorky, ktoré sú zhubné, ale model ich označil za nezhubné. Tento typ chyby je v medicíne najkritickejší.

*Report klasifikácie:* Obsahuje metriky ako Precision, Recall a F1-Score pre každú triedu zvlášť, čo poskytuje hlbší pohľad na výkonnosť modelu.

Vizualizácia matice zámeny pomocou heatmapy nám pomôže lepšie a rýchlejšie pochopiť, kde sa model pomýlil.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. Vytvorenie predikcií na testovacích dátach
y_pred = model_lr.predict(X_test_scaled)

# 2. Vyhodnotenie modelu
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("--- Výsledky Logistickej Regresie ---")
print(f"Presnosť modelu (Accuracy): {accuracy:.4f}")

print("\nConfusion Matrix (Matica Zámeny):")
print(conf_matrix)

# Vizualizácia Confusion Matrix pre lepšie pochopenie
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=cancer.target_names, yticklabels=cancer.target_names)
plt.title('Confusion Matrix pre Logistickú Regresiu')
plt.xlabel('Predikovaná trieda')
plt.ylabel('Skutočná trieda')
plt.show()

print("\nClassification Report:")
print(class_report)

# 5. Overenie stability modelu (krížová validácia

Aj keď sú výsledky na jednej testovacej množine veľmi dobré, pre získanie skutočne spoľahlivého odhadu výkonu modelu je potrebné overiť jeho stabilitu. Môže sa totiž stať, že model dosiahol vysokú presnosť len vďaka "šťastnému" rozdeleniu dát.

Na overenie stability použijeme 5-násobnú krížovú validáciu (5-fold Cross-Validation). Pri tejto technike sa dáta rozdelia na 5 častí. Model sa natrénuje 5-krát, pričom zakaždým sa testuje na inej, dovtedy nevidenej pätine dát. Priemerná presnosť z týchto piatich testov nám poskytne oveľa robustnejší a dôveryhodnejší odhad výkonu. Nízka štandardná odchýlka výsledkov potvrdí, že model je konzistentný a spoľahlivý.

In [ ]:
# Import pre krížovú validáciu
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

# Načítanie a príprava dát (rovnaké ako predtým, ale bez rozdelenia)
cancer = load_breast_cancer()
X = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
y = pd.Series(cancer.target)

# Dôležité: Pre krížovú validáciu je tiež potrebné škálovať dáta
# V praxi by sa to robilo v rámci Pipeline, aby sa predišlo data leakage.
# Pre náš účel to urobíme raz na celom datasete pred krížovou validáciou.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Inicializácia modelu
model_lr_cv = LogisticRegression()

# Spustenie krížovej validácie
# 'cv' = 5 znamená, že rozdelíme dáta na 5 častí a model natrénujeme 5-krát
# 'scoring' = 'accuracy' hovorí, že chceme merať presnosť
scores = cross_val_score(model_lr_cv, X_scaled, y, cv=5, scoring='accuracy')

print("--- Výsledky Krížovej Validácie ---")
print(f"Výsledky pre jednotlivé testovacie sady (fold): {np.round(scores, 4)}")

# Priemerná presnosť a štandardná odchýlka
mean_accuracy = np.mean(scores)
std_dev = np.std(scores)

print(f"\nPriemerná presnosť (Cross-Validation Score): {mean_accuracy:.4f}")
print(f"Štandardná odchýlka presnosti: {std_dev:.4f}")

# 6. Interpretácia koeficientov modelu

Na rozdiel od komplexnejších modelov, akým je napríklad Random Forest, Logistická Regresia nám umožňuje priamo interpretovať vzťah medzi príznakmi a cieľovou premennou. To je v medicínskej diagnostike kľúčové, pretože nám to pomáha pochopiť, ktoré merania majú najväčší vplyv na predpoveď diagnózy.

**V tejto sekcii zanalyzujeme koeficienty modelu:**

*Koeficienty:* Každý príznak má priradený koeficient, ktorého absolútna hodnota ukazuje silu jeho vplyvu na predikciu. Čím je absolútna hodnota koeficientu vyššia, tým má príznak väčší vplyv.

**Znamienko koeficientu:**

*Kladný koeficient:* So zvyšujúcou sa hodnotou príznaku sa zvyšuje šanca, že ide o zhubný nádor.

*Záporný koeficient:* So zvyšujúcou sa hodnotou príznaku sa znižuje šanca, že ide o zhubný nádor.

Vizualizácia týchto koeficientov nám poskytne jasný a prehľadný pohľad na to, ktoré príznaky sú pre náš model najdôležitejšie.









In [ ]:
# Získanie koeficientov modelu
coefficients = model_lr.coef_[0]

# Získanie názvov príznakov
feature_names = cancer.feature_names

# Vytvorenie DataFrame pre lepšiu prehľadnosť
coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients
})
# Zotriedenie podľa absolútnej hodnoty koeficientov
coef_df['abs_coefficient'] = abs(coef_df['coefficient'])
coef_df = coef_df.sort_values(by='abs_coefficient', ascending=False)

print("--- Koeficienty Logistickej Regresie (Zoradené podľa vplyvu) ---")
print(coef_df[['feature', 'coefficient']].to_string(index=False))

# Vizualizácia koeficientov
plt.figure(figsize=(12, 8))
sns.barplot(x='coefficient', y='feature', data=coef_df, palette='coolwarm')
plt.title('Vplyv príznakov na predikciu (Logistická Regresia)')
plt.xlabel('Koeficient')
plt.ylabel('Príznak')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

# Interpretácia:
print("\n--- Interpretácia Koeficientov ---")
print("Koeficienty Logistickej regresie ukazujú, aký vplyv má daný príznak na šancu, že bude vzorka klasifikovaná ako zhubná (trieda 0).")
print("  - Kladný koeficient (napr. 'worst concave points'): So zvyšujúcou sa hodnotou tohto príznaku sa ZVYŠUJE šanca, že ide o zhubný nádor.")
print("  - Záporný koeficient (napr. 'worst symmetry'): So zvyšujúcou sa hodnotou tohto príznaku sa ZNIŽUJE šanca, že ide o zhubný nádor.")
print("\nNajväčší vplyv na predikciu má príznak s najvyššou absolútnou hodnotou koeficientu.")

# 7. Vytvorenie interaktívnej aplikácie Gradio

Aby sme demonštrovali praktické využitie nášho modelu, vytvoríme jednoduchú webovú aplikáciu pomocou knižnice Gradio. Aplikácia umožní používateľovi (napríklad lekárovi) zadávať hodnoty jednotlivých príznakov a v reálnom čase získať predikciu, či je nádor zhubný alebo nezhubný.

Táto aplikácia slúži ako nástroj na podporu rozhodovania a predstavuje finálny, profesionálny výstup celého projektu.

**Aplikácia bude:**

Brať na vstup 30 číselných hodnôt príznakov.

Automaticky škálovať vstupné dáta tak, ako sme to robili pri trénovaní modelu.

Vracať predpoveď diagnózy a jej pravdepodobnosť.



In [ ]:
import gradio as gr
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

# Načítanie celého datasetu pre získanie názvov príznakov
cancer = load_breast_cancer()
feature_names = cancer.feature_names
target_names = cancer.target_names

# Natrénovanie modelu a scaleru na CELOM datasete
# Pre účely aplikácie natrénujeme model na všetkých dostupných dátach,
# aby bol čo najpresnejší.
X = cancer.data
y = cancer.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model_lr = LogisticRegression()
model_lr.fit(X_scaled, y)

print("Model aj scaler boli úspešne natrénované a sú pripravené pre Gradio.")

In [ ]:
# 1. Definovanie hlavnej predikčnej funkcie
def predict_diagnosis(*args):
    """
    Táto funkcia berie na vstupe všetky príznaky, škáluje ich a vracia predikciu.
    """
    # Prevedenie vstupných argumentov do NumPy poľa
    input_features = np.array(list(args)).reshape(1, -1)

    # Dôležité: Škálovanie vstupných dát pomocou natrénovaného scalera
    input_scaled = scaler.transform(input_features)

    # Vytvorenie predikcie a pravdepodobnosti
    prediction = model_lr.predict(input_scaled)
    probability = model_lr.predict_proba(input_scaled)[0]

    # Prevod predikcie na zrozumiteľný text
    result_text = f"Predikcia modelu: **{target_names[prediction[0]].capitalize()}**"

    # Pridanie pravdepodobnosti
    if prediction[0] == 0:
        result_text += f"\n(Pravdepodobnosť zhubnosti: {probability[0]:.2f})"
    else:
        result_text += f"\n(Pravdepodobnosť nezhubnosti: {probability[1]:.2f})"

    return result_text

# 2. Vytvorenie vstupných komponentov
inputs = []
for feature_name in feature_names:
    inputs.append(gr.Number(label=feature_name, value=np.mean(df[feature_name]))) # Nastavíme priemernú hodnotu ako predvolenú

# 3. Vytvorenie rozhrania Gradio
interface = gr.Interface(
    fn=predict_diagnosis,
    inputs=inputs,
    outputs=gr.Textbox(label="Výsledok Diagnózy", lines=2),
    title="Klasifikačný Nástroj pre Diagnózu Rakoviny Prsníka",
    description="Vložte hodnoty meraní bunkových jadier pre získanie predikcie, či je nádor zhubný (malignant) alebo nezhubný (benign). Tento nástroj slúži ako podpora rozhodovania a nenahrádza lekársku diagnózu."
)

# 4. Spustenie aplikácie
interface.launch()

# 8. Záver a zhrnutie

Tento projekt úspešne demonštroval, ako je možné použiť Logistickú Regresiu na riešenie medicínskeho klasifikačného problému s vysokou presnosťou a spoľahlivosťou.

**Naše zistenia možno zhrnúť do troch kľúčových bodov:**

*Vynikajúca výkonnosť:* Model dosiahol priemernú presnosť takmer 98%, čo je pre diagnostický nástroj mimoriadne vysoká hodnota. Krížová validácia navyše potvrdila, že tento výkon je stabilný a robustný, nie náhodný.

*Minimalizácia kritických chýb:* Model preukázal mimoriadne nízky počet kritických chýb typu False Negative, čo znamená, že len zriedka klasifikoval zhubný nádor ako nezhubný. V medicíne je toto najdôležitejší aspekt spoľahlivosti.

*Interpretovateľnosť:* Vďaka analýze koeficientov Logistickej Regresie sme boli schopní identifikovať, ktoré príznaky majú najväčší vplyv na predikciu. To zvyšuje dôveru v model a jeho použiteľnosť v reálnej praxi.

Tento projekt slúži ako dôkaz, že aj relatívne jednoduché, no dobre pochopené a aplikované modely strojového učenia, môžu poskytnúť cenné a praktické výsledky, ktoré slúžia ako nástroj na podporu rozhodovania v dôležitých oblastiach, ako je medicína.